## Getting boxscore using nba_api

In [ ]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""


# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
from nba_api.stats.static import players


player_dict = players.get_players()
bron = [player for player in player_dict if player['full_name'] == 'LeBron James'][0]

# game ID base value for regular season games played in the 2022-2023 season
gameID_base = '00222{}'

# test boxscore variable
test_box = boxscore.BoxScore('0022200001')

# setting up a dataframe columns from statistics dictionary keys
box_keys = test_box.get_dict()['game']['homeTeam']['statistics'].keys()
print(test_keys)

# loop needs to 
# iterates through each matchup of a season
# collects home and away box score data dictionaries
# appends dictionaries into dataframe
# use the dates of the boxscore dictionary 

# test loop 
for i in range(1,5):
    gameID = str(i).zfill(5)
    print(gameID)
    box_first_five = boxscore.BoxScore(gameID_base.format(gameID))
    date = box_first_five.get_dict()['game']['gameEt']
    print(date)
    
   # away_box = box_first_five.get_dict()['game']['awayTeam']['statistics']
   # home_box = box_first_five.get_dict()['game']['homeTeam']['statistics']


import pandas as pd
#create dataframes for these dictionaries
away_df = pd.DataFrame.from_dict(away_box, orient='index').T
away_df
home_df = pd.DataFrame.from_dict(home_box, orient='index').T

both_box_df = pd.DataFrame.from_dict()

#a new df that houses the average three point percentage of a each day
threePoint_df = pd.DataFrame() 

